# Download the data files

In [1]:
interactions_filename = 'INTERACTIONS.csv'
users_filename = 'USERS.csv'

In [2]:
!wget https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/customer11k.csv  -O USERS.csv
!wget https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/interactions.csv -O INTERACTIONS.csv

--2020-10-17 10:27:35--  https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/customer11k.csv
Resolving tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)... 52.219.32.196
Connecting to tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)|52.219.32.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456961 (446K) [text/csv]
Saving to: ‘USERS.csv’

USERS.csv           100%[===================>] 446.25K   489KB/s    in 0.9s    

2020-10-17 10:27:37 (489 KB/s) - ‘USERS.csv’ saved [456961/456961]

--2020-10-17 10:27:37--  https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/interactions.csv
Resolving tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)... 52.219.32.196
Connecting to tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)|52.219.32.196|:

# Organize Python imports

In [3]:
import boto3
import json
import numpy as np
import pandas as pd
import csv
import string 
import time


# Describe the downloaed data file info

In [4]:
users_df=pd.read_csv("USERS.csv")
interactions_df=pd.read_csv("INTERACTIONS.csv")

In [5]:
print(users_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   USER_ID    11162 non-null  int64 
 1   deposit    11162 non-null  object
 2   age        11162 non-null  int64 
 3   job        11162 non-null  object
 4   marital    11162 non-null  object
 5   education  11162 non-null  object
dtypes: int64(2), object(4)
memory usage: 523.3+ KB
None


In [6]:
print(interactions_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32220 entries, 0 to 32219
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   USER_ID    32220 non-null  int64 
 1   ITEM_ID    32220 non-null  object
 2   TIMESTAMP  32220 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 755.3+ KB
None


In [7]:
users_df.head()

,USER_ID,deposit,age,job,marital,education
0,30908,yes,59,admin.,married,secondary
1,30909,yes,56,admin.,married,secondary
2,30910,yes,41,technician,married,secondary
3,30911,yes,55,services,married,secondary
4,30912,yes,54,admin.,married,tertiary


In [8]:
interactions_df.head()

,USER_ID,ITEM_ID,TIMESTAMP
0,35739,AC_NE,1579609263
1,35739,AC_AS,1580612023
2,35739,AC_PA,1581390495
3,35739,AC_AA,1582229479
4,38695,TA_NE,1575001446


# Upload data to S3

In [9]:
random_suffix = ''.join(np.random.choice(list(string.ascii_lowercase + string.digits), 5))
print(random_suffix)

uyroc


In [10]:
bucket = "personalize-connect-blog-data-"+random_suffix     # replace with the name of your S3 bucket
!aws s3 mb s3://{bucket}

make_bucket: personalize-connect-blog-data-uyroc


In [11]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')

In [12]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)

In [13]:
boto3.Session().resource('s3').Bucket(bucket).Object(users_filename).upload_file(users_filename)

# Create Personalize Service Artifacts using boto3 APIs

## Create Personalize schemas

In [14]:
users_schema_name='personalize-connect-blog-users-schema-'+random_suffix
interactions_schema_name='personalize-connect-blog-interaction-schema-'+random_suffix
print(users_schema_name)

personalize-connect-blog-users-schema-uyroc


In [15]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "deposit",
            "type": "string"
        },
        {
            "name": "age",
            "type": "long"
        },
        {
            "name": "job",
            "type": "string"
        },
        {
            "name": "marital",
            "type": "string"
        },
        {
            "name": "education",
            "type": "string"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = users_schema_name,
    schema = json.dumps(users_schema)
)

users_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:550768952848:schema/personalize-connect-blog-users-schema-uyroc",
  "ResponseMetadata": {
    "RequestId": "9a62f2cc-fddd-4783-a716-fed5da6403f6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:27:45 GMT",
      "x-amzn-requestid": "9a62f2cc-fddd-4783-a716-fed5da6403f6",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [16]:
interactions_schema =  {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = interactions_schema_name,
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:550768952848:schema/personalize-connect-blog-interaction-schema-uyroc",
  "ResponseMetadata": {
    "RequestId": "cf0e1117-5f61-4a34-ab5b-ab8462507cfe",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:27:45 GMT",
      "x-amzn-requestid": "cf0e1117-5f61-4a34-ab5b-ab8462507cfe",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create Dataset Group and Datasets

In [17]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-connect-blog-dataset-"+random_suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:550768952848:dataset-group/personalize-connect-blog-dataset-uyroc",
  "ResponseMetadata": {
    "RequestId": "82dd6e14-0b86-45a1-975d-80e5aaad748d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:27:45 GMT",
      "x-amzn-requestid": "82dd6e14-0b86-45a1-975d-80e5aaad748d",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [18]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


In [19]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn,
    name = "personalize-connect-blog-interaction-dataset-"+random_suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:us-east-1:550768952848:dataset/personalize-connect-blog-dataset-uyroc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "2814558f-68cb-4880-b0d1-b5b91cad4332",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:28:45 GMT",
      "x-amzn-requestid": "2814558f-68cb-4880-b0d1-b5b91cad4332",
      "content-length": "119",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [20]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn,
    name = "personalize-connect-blog-user-dataset-"+random_suffix
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:us-east-1:550768952848:dataset/personalize-connect-blog-dataset-uyroc/USERS",
  "ResponseMetadata": {
    "RequestId": "8517a2e5-34e2-4850-905c-f0ffa19dd694",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:28:45 GMT",
      "x-amzn-requestid": "8517a2e5-34e2-4850-905c-f0ffa19dd694",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Add S3 bucket policy for allowing access from personalize service

In [21]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

## Use the Personalize IAM Role 

In [22]:
from sagemaker import get_execution_role
import sagemaker

sagemaker_session = sagemaker.Session()
role_arn = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

## Create Data Import Job

In [23]:
print(bucket)

personalize-connect-blog-data-uyroc


In [24]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "interactions-dataset-import-"+random_suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
        
    }, roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:550768952848:dataset-import-job/interactions-dataset-import-uyroc",
  "ResponseMetadata": {
    "RequestId": "55af5fbb-124f-4352-9aaf-5701a2005b82",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:28:46 GMT",
      "x-amzn-requestid": "55af5fbb-124f-4352-9aaf-5701a2005b82",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [25]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "users-dataset-import-"+random_suffix,
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, users_filename)
        
    }, roleArn = role_arn
)

users_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:550768952848:dataset-import-job/users-dataset-import-uyroc",
  "ResponseMetadata": {
    "RequestId": "66488547-3883-48aa-9da5-d95c1dc5ad63",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:28:46 GMT",
      "x-amzn-requestid": "66488547-3883-48aa-9da5-d95c1dc5ad63",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [26]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = users_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [27]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
#     print(dataset_import_job)
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [28]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


In [29]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Create a Solution Defition in Personalize Service

In [30]:
create_solution_response = personalize.create_solution(
    name = 'predict-ci-hrnn-solution-'+ random_suffix ,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:550768952848:solution/predict-ci-hrnn-solution-uyroc",
  "ResponseMetadata": {
    "RequestId": "11f9d007-e061-420e-82ab-ee0538d4ab37",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:45:47 GMT",
      "x-amzn-requestid": "11f9d007-e061-420e-82ab-ee0538d4ab37",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:550768952848:solution/predict-ci-hrnn-solution-uyroc/0dce3545",
  "ResponseMetadata": {
    "RequestId": "edf0769e-c2dc-4486-a278-41067128c816",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 10:45:48 GMT",
      "x-amzn-requestid": "edf0769e-c2dc-4486-a278-41067128c816",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [32]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## Deploy & Create Campaign URL

In [33]:
create_campaign_response = personalize.create_campaign(
    name = "predict-ci-hrnn-campaign-"+random_suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,  
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:550768952848:campaign/predict-ci-hrnn-campaign-uyroc",
  "ResponseMetadata": {
    "RequestId": "a38f5e55-8790-4f05-9bac-0a8a6eef2a1d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 17 Oct 2020 11:27:52 GMT",
      "x-amzn-requestid": "a38f5e55-8790-4f05-9bac-0a8a6eef2a1d",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [34]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


# Get Customer Intent Predicted 

## Prediction for existing data

In [35]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '41250' )

# 35071
# 37604
# 36720
# 37003

print("Recommended items - score ")
for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - score 
LI_SW 0.9706938
HL_NE 0.0046203
LI_NE 0.0043103
TA_NE 0.0034559
AC_NE 0.0031928
FD_NE 0.0029376
LI_PA 0.0026807
CC_NE 0.002614
FD_CS 0.0017003
LI_CS 0.0011588
FD_PW 0.0005389
AC_AA 0.0004247
CC_AA 0.0001878
LI_AS 0.00018
HL_RP 0.0001453
HL_AS 0.0001448
FD_AD 0.0001293
AC_CS 0.0001256
FD_NA 0.0001213
AC_PA 0.0001101
HL_CS 0.0001019
CC_CA 9.98e-05
CC_CS 9.79e-05
AC_AS 6.01e-05
TA_CS 5.8e-05


## Prediction for new data

Using personalize APIs for updating user interaction data in real-time we can get recommendations after new events has heppened. This is done using the put_events API offered by Amazon Personalize service.

In [36]:
event_tracker = 'CallInentTracker'
response = personalize.create_event_tracker(
    name=event_tracker,
    datasetGroupArn=dataset_group_arn
)

tracking_id = response['trackingId']
print(response)

{'eventTrackerArn': 'arn:aws:personalize:us-east-1:550768952848:event-tracker/59a74a4d', 'trackingId': '950c6e4d-eb18-43e3-a0db-8858a3ea3024', 'ResponseMetadata': {'RequestId': '198c4abd-2a20-4a1d-9eb5-1f42d0e8ed98', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sat, 17 Oct 2020 11:36:53 GMT', 'x-amzn-requestid': '198c4abd-2a20-4a1d-9eb5-1f42d0e8ed98', 'content-length': '139', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [37]:
import random
import string

personalize_events = boto3.client(service_name='personalize-events')

letters = string.ascii_lowercase
session_id= ''.join(random.choice(letters) for i in range(10)) 

In [38]:
personalize_events.put_events(
    trackingId = tracking_id,
    userId= '35739',
    sessionId = session_id,
    eventList = [{
        'sentAt': int(time.time()-604800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"AC_NE\"}"
        },{
        'sentAt': int(time.time()-404800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"AC_AS\"}"
        },{
        'sentAt': int(time.time()-94800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"AC_PA\"}"
        }]
)

{'ResponseMetadata': {'RequestId': 'f7913461-2871-426f-b396-6788bbb600bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Sat, 17 Oct 2020 11:36:53 GMT',
   'x-amzn-requestid': 'f7913461-2871-426f-b396-6788bbb600bb',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [39]:
personalize_events.put_events(
    trackingId = tracking_id,
    userId= '88888',
    sessionId = session_id,
    eventList = [{
        'sentAt': int(time.time()-86400),
        'eventType': 'call',
        'properties': "{\"itemId\": \"HL_NE\"}"
        }]
)

{'ResponseMetadata': {'RequestId': 'd23d62ad-382c-411c-9a2b-37b4372d2fa9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Sat, 17 Oct 2020 11:36:53 GMT',
   'x-amzn-requestid': 'd23d62ad-382c-411c-9a2b-37b4372d2fa9',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [40]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '88888' )

print("Recommended items - scores")

for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - scores
TA_NE 0.2028039
FD_NE 0.1788397
CC_NE 0.1761407
AC_NE 0.1586923
LI_NE 0.1393879
HL_NE 0.135718
LI_AS 0.0015252
FD_NA 0.0009235
AC_AS 0.0007139
HL_DS 0.0006044
AC_AA 0.00053
FD_PW 0.0004566
HL_RP 0.0004283
LI_SW 0.0004081
TA_CS 0.0003357
CC_LC 0.0003179
HL_CS 0.0003176
CC_CA 0.0002656
FD_AD 0.0002376
LI_CS 0.0002111
FD_CS 0.0001741
AC_PA 0.000154
CC_CS 0.00013
LI_PA 0.0001289


In [41]:
campaign_arn

'arn:aws:personalize:us-east-1:550768952848:campaign/predict-ci-hrnn-campaign-uyroc'

In [42]:
tracking_id

'950c6e4d-eb18-43e3-a0db-8858a3ea3024'